In [ ]:
import pandas as pd
import numpy as np
import os

Data is supposed to be in a directory `Data/` containing subdirectories `annonceur1/` and `annonceur2` that contain the corresponding files.

## 1. Reduce memory usage
`reduce_mem_usage` function for reducing memory usage by downgrading data types:

source : https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65

In [2]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print('\n')
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        print('col', col)
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

## 2. Generate HDF

HDF format allows storing of mutliple pd.DataFrame objects into one file, each pd.DataFrame is identified by a unique key

In the following example, data from annonceur{i} is gathered into `annonceur{i}.hdf`

In [3]:
DATA_DIR = "/Users/LL/Documents/Centrale/GTA/OMA/1000mercis/Analyses/Data"
missing_periods_data = {}

DATA_ALIAS = {
    'annonceur1/annonceur1_campaign1_visite_2pages.csv': "a1c1",
    'annonceur1/annonceur1_campaign2_visite_2pages.csv': "a1c2",
    'annonceur1/annonceur1_campaign3_visite_2pages.csv': "a1c3",
    'annonceur1/annonceur1_campaign4_visite_2pages.csv': "a1c4",
    'annonceur2/annonceur2_campaign1_achat.csv': "a2c1achat",
    'annonceur2/annonceur2_campaign1_visite_page_produit.csv': "a2c1produit",
    'annonceur2/annonceur2_campaign1_visite_panier.csv': "a2c1panier",
}

In [45]:
for file in [
    'annonceur1/annonceur1_campaign1_visite_2pages.csv',
#     'annonceur1/annonceur1_campaign1_visite_engagee.csv',
    'annonceur1/annonceur1_campaign2_visite_2pages.csv',
#     'annonceur1/annonceur1_campaign2_visite_engagee.csv',
    'annonceur1/annonceur1_campaign3_visite_2pages.csv',
#     'annonceur1/annonceur1_campaign3_visite_engagee.csv',
    'annonceur1/annonceur1_campaign4_visite_2pages.csv',
#     'annonceur1/annonceur1_campaign4_visite_engagee.csv'
]:
    key = file.replace('annonceur1/', '').replace('.csv', '')
    df = pd.read_csv(f'{DATA_DIR}/{file}', usecols=[1, 2, 3, 4])
#     reduce_mem_usage(df)
#     df.to_hdf('Data/annonceur1/annonceur1.hdf', key=key, complevel=9)
    # Also create daily hdf
    df.loc[:, 'date'] = pd.to_datetime(
        df['impression_date'].str.split(' ').str[0],
        format='%Y-%m-%d'
    )
    daily = df.groupby(['date', 'group', 'view', 'is_conv']).size().rename('count')
    
    # on enleve grosses plages de donnees manquantes
    dates = daily.reset_index()["date"].drop_duplicates()
    dates = pd.Series(dates.values, index=dates.values)
    dates_diff = dates.diff(1).loc[lambda x: x > pd.Timedelta("1D")]
    missing_periods = []
    for idx, delta in dates_diff.iteritems():
        missing_periods.append((
            (idx - delta + pd.Timedelta("1D")).strftime("%Y-%m-%d"), 
            (idx - pd.Timedelta("1D")).strftime("%Y-%m-%d"),
            delta - pd.Timedelta("1D")
        ))
    print("Missing periods for ", file)
    print(missing_periods)
    missing_periods_data[file] = missing_periods
    if file in [
        'annonceur1/annonceur1_campaign1_visite_2pages.csv',
        'annonceur1/annonceur1_campaign2_visite_2pages.csv',
        'annonceur1/annonceur1_campaign4_visite_2pages.csv',
    ]:
        # on enleve plages avec trop de donnees manquantes
        daily = daily.loc[:"20180430"]
        
    daily.to_hdf(f'{DATA_DIR}/annonceur1/annonceur1_daily.hdf', key=key)

Missing periods for  annonceur1/annonceur1_campaign1_visite_2pages.csv
[('2018-05-01', '2018-05-20', Timedelta('20 days 00:00:00'))]
Missing periods for  annonceur1/annonceur1_campaign2_visite_2pages.csv
[('2018-02-22', '2018-02-25', Timedelta('4 days 00:00:00')), ('2018-05-01', '2018-05-20', Timedelta('20 days 00:00:00'))]
Missing periods for  annonceur1/annonceur1_campaign3_visite_2pages.csv
[('2018-03-13', '2018-03-13', Timedelta('1 days 00:00:00'))]
Missing periods for  annonceur1/annonceur1_campaign4_visite_2pages.csv
[('2018-05-01', '2018-05-20', Timedelta('20 days 00:00:00'))]


> Pour a1c1, a1c2, a1c4, on enlève les dates après 20180430. Notons qu'il restera des plages à trous pour a2c1 de 20180222 à 20180225 et pour a1c3 il manque la journée du 20180313.

In [17]:
for file in [
    'annonceur2/annonceur2_campaign1_achat.csv',
    'annonceur2/annonceur2_campaign1_visite_page_produit.csv',
    'annonceur2/annonceur2_campaign1_visite_panier.csv'
]:
    key = file.replace('annonceur2/', '').replace('.csv', '')
    df = pd.read_csv(f'{DATA_DIR}/{file}', usecols=[1, 2, 3, 4])
#     reduce_mem_usage(df)
#     df.to_hdf('Data/annonceur2/annonceur2.hdf', key=key, complevel=9)
    
    # Also create daily hdf
    df.loc[:, 'date'] = pd.to_datetime(
        df['impression_date'].str.split(' ').str[0],
        format='%Y-%m-%d'
    )
    daily = df.groupby(['date', 'group', 'view', 'is_conv']).size().rename('count')
    
    # on enleve grosses plages de donnees manquantes
    dates = daily.reset_index()["date"].drop_duplicates()
    dates = pd.Series(dates.values, index=dates.values)
    dates_diff = dates.diff(1).loc[lambda x: x > pd.Timedelta("1D")]
    missing_periods = []
    for idx, delta in dates_diff.iteritems():
        missing_periods.append((
            (idx - delta + pd.Timedelta("1D")).strftime("%Y-%m-%d"), 
            (idx - pd.Timedelta("1D")).strftime("%Y-%m-%d"),
            delta - pd.Timedelta("1D")
        ))
    print("Missing periods for ", file)
    print(missing_periods)
    missing_periods_data[file] = missing_periods
    if file in []:
        # on enleve plages avec trop de donnees manquantes
        pass
    
    daily.to_hdf(f'{DATA_DIR}/annonceur2/annonceur2_daily.hdf', key=key)

Missing periods for  annonceur2/annonceur2_campaign1_achat.csv
[]
Missing periods for  annonceur2/annonceur2_campaign1_visite_page_produit.csv
[]
Missing periods for  annonceur2/annonceur2_campaign1_visite_panier.csv
[]


In [56]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly

plotly.offline.init_notebook_mode(connected=True)

gantt_data = []
for file, missing_periods in missing_periods_data.items():
    for period in missing_periods:  # ok si de longueur 0
        gantt_data.append(
            {"Task": DATA_ALIAS[file], "Start": period[0], "Finish": period[1], "MissingDays": period[2].days}
        )
        

fig = ff.create_gantt(
    gantt_data, 
    show_colorbar=True, 
    group_tasks=True, 
    index_col="MissingDays",
    title="Jours manquants par campagne"
)

plotly.offline.iplot(fig)

> Attention on ne voit pas qu'il y a un jour manquant (20180313) pour a1c3.

## 3. Example for loading one particular DataFrame :

Load brut data :

In [ ]:
df = pd.read_hdf(f'{DATA_DIR}/annonceur2/annonceur2.hdf', key='annonceur2_campaign1_visite_panier')

In [ ]:
df.info()

Load only daily (aggregated) data :

In [5]:
key = 'annonceur1_campaign1_visite_2pages'
key = 'annonceur1_campaign1_visite_engagee'
key = 'annonceur1_campaign2_visite_2pages'
# key = 'annonceur1_campaign2_visite_engagee'
# key = 'annonceur1_campaign3_visite_2pages'
# key = 'annonceur1_campaign3_visite_engagee'
key = 'annonceur1_campaign4_visite_2pages'
# key = 'annonceur1_campaign4_visite_engagee'


daily = pd.read_hdf(f'{DATA_DIR}/annonceur1/annonceur1_daily.hdf', key=key)

In [ ]:
daily

In [ ]:
daily = pd.read_hdf(f'{DATA_DIR}/annonceur2/annonceur2_daily.hdf', key='annonceur2_campaign1_visite_panier')